<hr>
# 第7讲-网络爬虫常见技巧

1. 模拟登录
2. 常见验证码突破
3. 爬虫代理池
4. SSL认证问题
5. 设计健壮的爬虫
6. 各种爬虫的坑
7. 爬虫攻防小结

## 1. 模拟登录

### 1.1常规模拟登录


In [ ]:
import urllib
try:
    # python 2
    import cookielib
except:
    # python 3
    import http.cookiejar as cookielib
        
from bs4 import BeautifulSoup

In [ ]:
import urllib
import http.cookiejar as cookielib
from bs4 import BeautifulSoup
filename = 'cookieOfCsdn.txt'
cookie = cookielib.MozillaCookieJar(filename)
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)
loginurl = 'https://passport.csdn.net/account/login?from=http%3A%2F%2Fmy.csdn.net%2Fmy%2Fmycsdn'
response = opener.open(loginurl)
soup = BeautifulSoup(response.read(), 'html.parser')
for input in soup.form.find_all('input'):
    if input.get("name") == "lt":
        lt = input.get("value")
    if input.get("name") == "execution":
        execution = input.get("value")
values = {
     'username':'username',
     'password':'passwd',
     'lt':lt,
     'execution':execution,
     '_eventId':'submit'
 }
postdata = urllib.parse.urlencode(values).encode(encoding='UTF8')
print(postdata)
opener.addheaders = [("User-Agent", "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36")]

result = opener.open(loginurl, postdata)
cookie.save(ignore_expires=True, ignore_discard=True)
url1 = "http://my.csdn.net/my/mycsdn"
result1 = opener.open(url1)
print(result1.read().decode('utf-8'))

### 1.2利用selenium做模拟登录

案例：微信公众号信息抓取

https://wx.qq.com/


1. 通过Selenium调用Chrome登录web版微信
2. 模拟登录
3. 点击切换到公众号面板
4. 爬取公众号数据
5. 定时刷新页面

In [1]:
from selenium import webdriver

In [2]:
browser = webdriver.Chrome()

In [3]:
browser.get('https://wx.qq.com/')

In [ ]:
#2. 模拟登录
#3. 点击切换到公众号面板

In [1]:
print(browser.page_source)

In [ ]:
browser.quit()

In [ ]:
#5. 定时刷新页面技巧

import time  
from selenium import webdriver  
  
browser = webdriver.Chrome()  
browser.maximize_window()  
browser.implicitly_wait(6)  
  
browser.get("https://www.baidu.com")  
time.sleep(3)  
try:  
    browser.refresh()
    print('刷新成功')  
except Exception as e:  
    print("Error", format(e))
    
browser.quit()  

## 2. 常见验证码突破

In [ ]:
# 1. pytesseract图像识别
# 2. 基于机器学习的图像识别
# 3. 基于深度学习的图像识别

## 3. 爬虫代理池

In [1]:
# 1. User-Agent 更换
# 2. ip更换

In [2]:
# 1. User-Agent 更换

#http://hexingxing.cn/ua-user-agent-browser/
#user agent大全

In [ ]:
import requests
import re
import random

user_agent_list = [
            "Mozilla/5.0",
            "Mozilla/5.1"
]

user_agent = random.choice(user_agent_list) 
headers = {'User-Agent': user_agent}
response = requests.get(url, headers=headers)

In [3]:
# 2. ip更换
# ip池

In [ ]:
# http://www.xicidaili.com/nn/

In [ ]:
def get_url(url):     # 获取免费代理ip
    url_list = []
    for i in range(1,100):
        url_new = url + str(i)
        url_list.append(url_new)
    return url_list

In [ ]:
def get_content(url):     # 免费代理ip的每个页面
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36'}
    req = urllib.request.Request(url=url, headers=headers)
    res = urllib.request.urlopen(req)
    content = res.read()
    return content.decode('utf-8')

In [ ]:
def get_info(content):      # 获取代理ip内容
    ip = etree.HTML(content).xpath('//table[contains(@id,"ip_list")]/tr/td[2]/text()')
    port = etree.HTML(content).xpath('//table[contains(@id,"ip_list")]/tr/td[3]/text()')
    with open("data.txt", "w") as f:
        for i in range(0,len(ip)):
            out = ""
            out += "" + ip[i]
            out += ":" + port[i]
            f.write(out + "\n")     # 写入data.txt文件

In [ ]:
def verify_ip(ip,port):    # 验证爬下来的ip有效性
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36'}
    
    proxy = {'http':'http://%s:%s'%(ip,port)}
    print(proxy)

    proxy_handler = urllib.request.ProxyHandler(proxy)
    opener = urllib.request.build_opener(proxy_handler)
    urllib.request.install_opener(opener)

    test_url = "https://www.baidu.com/"
    req = urllib.request.Request(url=test_url,headers=headers)
    time.sleep(6)
    try:
        res = urllib.request.urlopen(req)
        time.sleep(3)
        content = res.read()
        if content:
            print('ip可用')
            with open("data_verify.txt", "a") as f:
                f.write(ip + ":" + port)
                f.write("\n")
        else:
            print('ip不可用')
    except urllib.request.URLError as e:
        print(e.reason)

In [ ]:
import urllib
import time
from lxml import etree

url = 'http://www.xicidaili.com/nn/'
url_list = get_url(url)
for i in url_list:
    print(i)
    content = get_content(i)
    time.sleep(3)
    get_info(content)
    
with open("data.txt", "r") as f:
    data_list = f.readlines()
    for data in data_list:
        print(data.split(":")[0])
        verify_ip(data.split(":")[0],data.split(":")[1])
        

## 4. SSL认证问题

https://wandergis.com/hospital-viz/

In [ ]:
a = requests.get('https://wandergis.com/hospital-viz/', headers=headers)

SSL证书是数字证书的一种，类似于驾驶证、护照和营业执照的电子副本。因为配置在服务器上，也称为SSL服务器证书。SSL 证书就是遵守 SSL协议，由受信任的数字证书颁发机构CA，在验证服务器身份后颁发，具有服务器身份验证和数据传输加密功能

> SSL Cert Verification Requests verifies SSL certificates for HTTPS requests, just like a web browser. By default, SSL verification is enabled, and Requests will throw a SSLError if it's unable to verify the certificate

http://docs.python-requests.org/en/latest/user/advanced/#ssl-cert-verification

In [ ]:
# get的verify=False参数

In [ ]:
#a.status_code
#a.json()

## 5. 设计健壮的爬虫

### 5.1 关于异常

- 由于爬虫常常遇到网站的各种情况，比如中断，页面更改等，我们在下载页面时，会做很多异常处理的动作，尽量保证下载不受到各种中断的影响。
    
- 下面按照简单到复杂，易干扰到健壮的方式，介绍关于异常的处理：

- 1.简单下载

In [ ]:
import urllib2
# 便于创建绝对路径
import urlparse


def download1(url):
    """简单直接下载"""
    return urllib2.urlopen(url).read()

- 2.捕获异常

In [ ]:
import urllib2
import urlparse

def download2(url):
    """可以捕获一般的网络异常 """
    print 'Downloading:', url
    try:
        html = urllib2.urlopen(url).read()
    except urllib2.URLError as e:
        print 'Download error:', e.reason
        html = None
    return html

- 3.下载重试

In [ ]:
import urllib2
import urlparse

def download3(url, num_retries=5):
    """如果捕获5XX（服务器状态）相关异常，可以进行重试操作"""
    print 'Downloading:', url
    try:
        html = urllib2.urlopen(url).read()
    except urllib2.URLError as e:
        print 'Download error:', e.reason
        html = None
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                # 进行重试连接
                html = download3(url, num_retries-1)
    return html

httpstat.us/500

- 4.添加UA

In [ ]:
import urllib2
import urlparse


def download4(url, user_agent='ua', num_retries=5):
    """添加user agent 连接"""
    print 'Downloading:', url
    headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36'}
    request = urllib2.Request(url, headers=headers)
    try:
        html = urllib2.urlopen(request).read()
    except urllib2.URLError as e:
        print 'Download error:', e.reason
        html = None
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                # 进行重试连接
                html = download4(url, user_agent, num_retries-1)
    return html

- 5.添加代理

In [ ]:
import urllib2
import urlparse

def download5(url, user_agent='ua', proxy=None, num_retries=5):
    """支持代理的连接方式"""
    print 'Downloading:', url
    headers = {'User-agent': 'Mozilla/5.0'}
    request = urllib2.Request(url, headers=headers)
    opener = urllib2.build_opener()
    if proxy:
        proxy_params = {urlparse.urlparse(url).scheme: proxy}
        opener.add_handler(urllib2.ProxyHandler(proxy_params))
    try:
        html = opener.open(request).read()
    except urllib2.URLError as e:
        print 'Download error:', e.reason
        html = None
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                # 进行重试连接
                html = download5(url, user_agent, proxy, num_retries-1)
    return html

实施：

In [ ]:
print download('http://example.webscraping.com')

-------------

## 6. 各种爬虫的坑

- 字符集编码问题（不要随意encode, decode；尽量统一字符集编码；了解web页面，本地，文件等编码格式）
- HTML/XML不规范（必要时候还是使用正则表达）
- 网页跳转（重定向，利用Selenium监听，找到实际数据存在页面）
- Cookie/认证会话维护（直接Cookie，模拟登陆）
- 代理维护（代理有效性测试，定期更新）
- 超时问题
- 内存优化
- 异常维护（代码持续运行，爬虫性能，健壮性要求）
- 代码框架的可扩展性

--------------------

## 7. 爬虫攻防小结

阻挡方式：
- 客户端请求的数据
- 客户端请求的方式
- 服务器端的数据组织和加载

相应策略：
- 在headers中添加合适的数据进行欺骗
- 使用代理服务
- 模拟登陆，模拟登出
- 控制爬取时间（robots.txt）
- 使用Selenium+PhantomJS类似的库或者工具

反爬的手段：

- 封禁
- 投毒（地理位置）
- 投毒（商家信息）